# Option Pricing with Dynamic Programming

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
def vector_to_spreadsheet(vector):
    N = int((np.sqrt(1+8*(vector.shape[0]-1))-1)/2)-1
    spreadsheet = pd.DataFrame((N+1)*[(N+1)*[np.nan]])
    for i in range(N+1):
        if i==0:
            spreadsheet.iloc[-1,0] = vector[0,0]
        else:
            for j in range(i+1):
                k = i*(i+1)//2 + j
                spreadsheet.iloc[-j-1,i] = vector[k]
    return spreadsheet.fillna('')

In [3]:
def calibrated_parameters(n, T, r, volatility, c):
    R = np.exp(r*T/n)
    u = np.exp(volatility*np.sqrt(T/n))
    v = 1/u
    q = (np.exp((r-c)*T/n)-v)/(u-v)
    return R, u, v, q

def binomial_model(n, S0, u, v):
    states = []
    for j in range(n+1):
        for i in range(j+1):
            state = S0*np.power(u,i)*np.power(v,j-i)
            states.append([state, j])
    states.append([np.inf, (n+1)])
    return np.array(states), ['HOLD','EXECUTE']

def call_payoff(K):
    def f(S):
        return np.maximum(S-K,0)
    return f

def put_payoff(K):
    def f(S):
        return np.maximum(K-S,0)
    return f

In [4]:
n = 15
T = 0.25
S0 = 100
r = 2./100
volatility = 30./100
K = 110
c = 1./100

R, u, v, q = calibrated_parameters(n=n, T=T, r=r, volatility=volatility, c=c)

In [5]:
states, actions = binomial_model(n, S0, u, v)
vector_to_spreadsheet(states[:,[0]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,178.773151
1,,,,,,,,,,,,,,,171.981662,165.448178
2,,,,,,,,,,,,,,165.448178,159.162897,153.116390
3,,,,,,,,,,,,,159.162897,153.11639,147.299587,141.703761
4,,,,,,,,,,,,153.11639,147.299587,141.703761,136.320517,131.141779
5,,,,,,,,,,,147.299587,141.703761,136.320517,131.141779,126.159778,121.367041
6,,,,,,,,,,141.703761,136.320517,131.141779,126.159778,121.367041,116.756377,112.320870
7,,,,,,,,,136.320517,131.141779,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961
8,,,,,,,,131.141779,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961,100.0,96.201058
9,,,,,,,126.159778,121.367041,116.756377,112.32087,108.053865,103.948961,100.0,96.201058,92.546435,89.030649


-----

In [6]:
def generate_probabilities(states, actions, q):
    N = int((np.sqrt(1+8*(len(states)-1))-1)/2)
    p = np.zeros(shape=(len(actions),len(states),len(states)))
    i = 0
    for t in range(N-1):
        for j in range((t+1)*(t+2)//2,t+1+(t+1)*(t+2)//2):
            p[0,i,j] = 1-q
            p[0,i,j+1] = q
            i = i+1
    for k in range(i,len(states)-1):
        p[0,k,-1] = 1
    p[0,-1,-1] = 1
    for i in range(1,len(actions)):
        p[i,:,-1] = 1
    return p

def generate_rewards(states, actions, payoff):
    N = int((np.sqrt(1+8*(len(states)-1))-1)/2)
    r = np.zeros(shape=(len(states),len(actions))) 
    r[-(N+1):-1,0] = payoff(np.array(states[-(N+1):-1,0])) 
    r[:-1,1] = payoff(np.array(states[:-1,0]))
    return r

-----

In [7]:
class MDP:
    
    def __init__(self, states, actions, probabilities, rewards, discount_factor):
        self.states = states
        self.actions = actions
        self.probabilities = probabilities
        self.rewards = rewards
        self.discount_factor = discount_factor
              
    def policy_evaluation(self, policy, theta, verbose=True):
        v_init = np.random.normal(loc=0.0, scale=0.05, size=(len(self.states),1))
        P_pi = np.einsum('axy,xa->xy', self.probabilities, policy)
        r_pi = np.einsum('xa,xa->x', self.rewards, policy).reshape(-1,1)
        while True:
            start_time = time.time()
            v = r_pi + self.discount_factor*np.dot(P_pi, v_init)
            error = np.max(np.abs(v-v_init))
            now = time.time()
            duration = now - start_time
            if verbose:
                print('Error: {}'.format(error))
                print('----- {}s -----'.format(np.round(1000*duration)/1000))
            if error<theta:
                break
            v_init = v
        return v
    
    def policy_improvement(self, v):
        q = self.rewards + self.discount_factor*np.transpose(np.einsum('axy,y->ax', self.probabilities, v.flatten()))
        idx = np.argmax(q, axis=1)
        return np.eye(len(self.actions))[idx]
        
    def value_iteration(self, theta, verbose=True):
        v_init = np.random.normal(loc=0.0, scale=0.05, size=(len(self.states),1))
        while True:
            start_time = time.time()
            q = self.rewards + self.discount_factor*np.transpose(np.einsum('axy,y->ax', self.probabilities, v_init.flatten()))
            v = np.max(q, axis=1, keepdims=True)
            error = np.max(np.abs(v-v_init))
            now = time.time()
            duration = now - start_time
            if verbose:
                print('Error: {}'.format(error))
                print('----- {}s -----'.format(np.round(1000*duration)/1000))
            if error<theta:
                break
            v_init = v
        self.value_star = v
        self.policy_star = self.policy_improvement(v=self.value_star)
    
    def policy_iteration(self, theta, verbose=True):        
        pi_init = np.ones((len(self.states), len(self.actions)))/len(self.actions)
        v_init = self.policy_evaluation(policy=pi_init, theta=theta, verbose=False) 
        while True:
            start_time = time.time()
            pi = self.policy_improvement(v_init)
            v = self.policy_evaluation(policy=pi, theta=theta, verbose=False)
            error = np.max(np.abs(v-v_init))
            now = time.time()
            duration = now - start_time
            if verbose:
                print('Error: {}'.format(error))
                print('----- {}s -----'.format(np.round(1000*duration)/1000))    
            if error<theta:
                break
            v_init = v
        self.value_star = v
        self.policy_star = pi

-----

In [8]:
p = generate_probabilities(states, actions, q)
r = generate_rewards(states, actions, call_payoff(K))
discount_factor = 1/R

In [9]:
mdp = MDP(states = states, 
          actions = actions, 
          probabilities = p, 
          rewards = r, 
          discount_factor = discount_factor)

In [10]:
policy = np.zeros((len(states), len(actions)))
N = int((np.sqrt(1+8*(len(states)-1))-1)/2)-1
for i in range(len(states)-1):
    if i<=N*(N+1)//2-1:
        policy[i, 0] = 1
    else:
        policy[i, -1] = 1
policy[-1, 0] = 1
policy

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [11]:
value_function = mdp.policy_evaluation(policy=policy, theta=1e-9)

Error: 68.76667914493116
----- 0.003s -----
Error: 61.99065457880374
----- 0.002s -----
Error: 55.46418926618769
----- 0.005s -----
Error: 49.189268743091475
----- 0.001s -----
Error: 43.15922228118383
----- 0.0s -----
Error: 37.360964729839274
----- 0.0s -----
Error: 31.782434503910277
----- 0.0s -----
Error: 26.4670417088853
----- 0.0s -----
Error: 21.5174405768769
----- 0.0s -----
Error: 17.05605478107972
----- 0.0s -----
Error: 13.1805925821601
----- 0.0s -----
Error: 9.938020430697735
----- 0.0s -----
Error: 7.320606140429809
----- 0.0s -----
Error: 5.27695826582339
----- 0.0s -----
Error: 3.7290051213319138
----- 0.0s -----
Error: 2.588268397545994
----- 0.0s -----
Error: 2.6927487297712105e-06
----- 0.0s -----
Error: 2.6918512929796634e-06
----- 0.0s -----
Error: 2.6909541546160654e-06
----- 0.0s -----
Error: 2.6900573146804163e-06
----- 0.0s -----
Error: 2.6891607802781436e-06
----- 0.0s -----
Error: 2.68826454430382e-06
----- 0.0s -----
Error: 2.687368599652018e-06
----- 0.0s 

Error: 1.979602039625661e-06
----- 0.0s -----
Error: 1.9789422793792255e-06
----- 0.0s -----
Error: 1.978282742953752e-06
----- 0.0s -----
Error: 1.977623433901954e-06
----- 0.0s -----
Error: 1.9769643344602628e-06
----- 0.0s -----
Error: 1.9763054552868198e-06
----- 0.0s -----
Error: 1.975646796381625e-06
----- 0.0s -----
Error: 1.9749883506392507e-06
----- 0.001s -----
Error: 1.974330132270552e-06
----- 0.0s -----
Error: 1.9736721341701013e-06
----- 0.0s -----
Error: 1.973014352785185e-06
----- 0.0s -----
Error: 1.972356791668517e-06
----- 0.0s -----
Error: 1.9716994472673832e-06
----- 0.0s -----
Error: 1.971042330239925e-06
----- 0.0s -----
Error: 1.97038541926986e-06
----- 0.0s -----
Error: 1.969728732120757e-06
----- 0.0s -----
Error: 1.9690722652399018e-06
----- 0.0s -----
Error: 1.968416015074581e-06
----- 0.0s -----
Error: 1.9677599851775085e-06
----- 0.0s -----
Error: 1.9671041826541114e-06
----- 0.0s -----
Error: 1.9664485861881076e-06
----- 0.0s -----
Error: 1.96579321354306

Error: 1.530451058329163e-06
----- 0.0s -----
Error: 1.5299409952262977e-06
----- 0.0s -----
Error: 1.5294311026536889e-06
----- 0.0s -----
Error: 1.5289213806113366e-06
----- 0.0s -----
Error: 1.5284118219938136e-06
----- 0.0s -----
Error: 1.5279024339065472e-06
----- 0.0s -----
Error: 1.5273932163495374e-06
----- 0.0s -----
Error: 1.5268841764282115e-06
----- 0.0s -----
Error: 1.5263752928262875e-06
----- 0.0s -----
Error: 1.5258665868600474e-06
----- 0.0s -----
Error: 1.5253580460949934e-06
----- 0.0s -----
Error: 1.524849686518337e-06
----- 0.0s -----
Error: 1.5243414814847256e-06
----- 0.0s -----
Error: 1.5238334540867982e-06
----- 0.0s -----
Error: 1.5233256078772683e-06
----- 0.0s -----
Error: 1.522817910881713e-06
----- 0.0s -----
Error: 1.5223103844164143e-06
----- 0.0s -----
Error: 1.5218030320340858e-06
----- 0.0s -----
Error: 1.5212958466293003e-06
----- 0.0s -----
Error: 1.5207888317547713e-06
----- 0.0s -----
Error: 1.5202819838577852e-06
----- 0.0s -----
Error: 1.5197753

Error: 1.1349203852262235e-06
----- 0.0s -----
Error: 1.1345421420116963e-06
----- 0.0s -----
Error: 1.134164023142148e-06
----- 0.0s -----
Error: 1.1337860357230056e-06
----- 0.0s -----
Error: 1.1334081690961284e-06
----- 0.001s -----
Error: 1.1330304303669436e-06
----- 0.0s -----
Error: 1.132652812430024e-06
----- 0.001s -----
Error: 1.132275329496224e-06
----- 0.0s -----
Error: 1.1318979673546892e-06
----- 0.0s -----
Error: 1.1315207260054194e-06
----- 0.0s -----
Error: 1.1311436196592695e-06
----- 0.0s -----
Error: 1.1307666341053846e-06
----- 0.0s -----
Error: 1.1303897764491921e-06
----- 0.0s -----
Error: 1.1300130431379785e-06
----- 0.0s -----
Error: 1.1296364377244572e-06
----- 0.0s -----
Error: 1.129259953103201e-06
----- 0.0s -----
Error: 1.12888358927421e-06
----- 0.0s -----
Error: 1.128507356895625e-06
----- 0.0s -----
Error: 1.1281312524147324e-06
----- 0.0s -----
Error: 1.127755268726105e-06
----- 0.0s -----
Error: 1.12737941293517e-06
----- 0.0s -----
Error: 1.1270036850

Error: 8.724608591137439e-07
----- 0.0s -----
Error: 8.721700837099888e-07
----- 0.0s -----
Error: 8.718794113349304e-07
----- 0.0s -----
Error: 8.715888348831413e-07
----- 0.0s -----
Error: 8.712983543546216e-07
----- 0.001s -----
Error: 8.710079697493711e-07
----- 0.0s -----
Error: 8.707176757383195e-07
----- 0.0s -----
Error: 8.704274847559645e-07
----- 0.0s -----
Error: 8.701373914732358e-07
----- 0.0s -----
Error: 8.698473905610626e-07
----- 0.0s -----
Error: 8.695574926775862e-07
----- 0.0s -----
Error: 8.692676871646654e-07
----- 0.0s -----
Error: 8.689779775750139e-07
----- 0.0s -----
Error: 8.686883674613455e-07
----- 0.0s -----
Error: 8.6839885682366e-07
----- 0.0s -----
Error: 8.681094385565302e-07
----- 0.0s -----
Error: 8.678201197653834e-07
----- 0.001s -----
Error: 8.675308897920786e-07
----- 0.0s -----
Error: 8.672417628474705e-07
----- 0.0s -----
Error: 8.669527247207043e-07
----- 0.0s -----
Error: 8.666637967280622e-07
----- 0.0s -----
Error: 8.663749540005483e-07
---

...

Error: 3.960298755600888e-09
----- 0.0s -----
Error: 3.958987804253411e-09
----- 0.0s -----
Error: 3.957666194764897e-09
----- 0.0s -----
Error: 3.9563445852763834e-09
----- 0.0s -----
Error: 3.95502297578787e-09
----- 0.0s -----
Error: 3.9537084717267135e-09
----- 0.0s -----
Error: 3.9523868622382e-09
----- 0.001s -----
Error: 3.9510723581770435e-09
----- 0.0s -----
Error: 3.94975074868853e-09
----- 0.0s -----
Error: 3.9484362446273735e-09
----- 0.0s -----
Error: 3.947128845993575e-09
----- 0.0s -----
Error: 3.945807236505061e-09
----- 0.0s -----
Error: 3.944492732443905e-09
----- 0.0s -----
Error: 3.94317467566907e-09
----- 0.0s -----
Error: 3.941860171607914e-09
----- 0.0s -----
Error: 3.940556325687794e-09
----- 0.0s -----
Error: 3.939241821626638e-09
----- 0.0s -----
Error: 3.937927317565482e-09
----- 0.0s -----
Error: 3.9366128135043255e-09
----- 0.0s -----
Error: 3.935298309443169e-09
----- 0.0s -----
Error: 3.933990910809371e-09
----- 0.0s -----
Error: 3.932683512175572e-09
---

Error: 3.022204708713616e-09
----- 0.0s -----
Error: 3.021206396169873e-09
----- 0.0s -----
Error: 3.020197425485094e-09
----- 0.0s -----
Error: 3.0191884548003145e-09
----- 0.0s -----
Error: 3.0181865895428928e-09
----- 0.0s -----
Error: 3.0171776188581134e-09
----- 0.001s -----
Error: 3.0161757536006917e-09
----- 0.0s -----
Error: 3.0151667829159123e-09
----- 0.0s -----
Error: 3.014161364944812e-09
----- 0.0s -----
Error: 3.0131523942600325e-09
----- 0.0s -----
Error: 3.0121540817162895e-09
----- 0.0s -----
Error: 3.01114511103151e-09
----- 0.0s -----
Error: 3.0101432457740884e-09
----- 0.0s -----
Error: 3.0091413805166667e-09
----- 0.0s -----
Error: 3.008139515259245e-09
----- 0.0s -----
Error: 3.007137650001823e-09
----- 0.0s -----
Error: 3.006128679317044e-09
----- 0.0s -----
Error: 3.005126814059622e-09
----- 0.0s -----
Error: 3.0041249488022004e-09
----- 0.0s -----
Error: 3.0031266362584574e-09
----- 0.0s -----
Error: 3.0021247710010357e-09
----- 0.0s -----
Error: 3.001126458457

Error: 2.3521309344687324e-09
----- 0.0s -----
Error: 2.351356442886754e-09
----- 0.0s -----
Error: 2.350571293163739e-09
----- 0.0s -----
Error: 2.3497825907270453e-09
----- 0.0s -----
Error: 2.3489974410040304e-09
----- 0.0s -----
Error: 2.3482158439946943e-09
----- 0.001s -----
Error: 2.347434246985358e-09
----- 0.0s -----
Error: 2.346652649976022e-09
----- 0.0s -----
Error: 2.345871052966686e-09
----- 0.0s -----
Error: 2.34508945595735e-09
----- 0.0s -----
Error: 2.3443078589480137e-09
----- 0.0s -----
Error: 2.343522709224999e-09
----- 0.0s -----
Error: 2.3427446649293415e-09
----- 0.0s -----
Error: 2.3419595152063266e-09
----- 0.0s -----
Error: 2.3411814709106693e-09
----- 0.0s -----
Error: 2.340406979328691e-09
----- 0.0s -----
Error: 2.3396253823193547e-09
----- 0.0s -----
Error: 2.3388437853100186e-09
----- 0.0s -----
Error: 2.3380657410143613e-09
----- 0.0s -----
Error: 2.337284144005025e-09
----- 0.004s -----
Error: 2.336506099709368e-09
----- 0.0s -----
Error: 2.33572805541

Error: 1.774743907390075e-09
----- 0.0s -----
Error: 1.7741541569193942e-09
----- 0.003s -----
Error: 1.7735608537350345e-09
----- 0.0s -----
Error: 1.7729675505506748e-09
----- 0.0s -----
Error: 1.772377800079994e-09
----- 0.0s -----
Error: 1.771788049609313e-09
----- 0.0s -----
Error: 1.7711982991386321e-09
----- 0.0s -----
Error: 1.7706085486679513e-09
----- 0.0s -----
Error: 1.7700116927699128e-09
----- 0.0s -----
Error: 1.7694290477265895e-09
----- 0.0s -----
Error: 1.7688357445422298e-09
----- 0.0s -----
Error: 1.7682495467852277e-09
----- 0.0s -----
Error: 1.7676597963145468e-09
----- 0.0s -----
Error: 1.7670735985575448e-09
----- 0.0s -----
Error: 1.7664838480868639e-09
----- 0.0s -----
Error: 1.7658905449025042e-09
----- 0.0s -----
Error: 1.7653007944318233e-09
----- 0.0s -----
Error: 1.7647181493885e-09
----- 0.0s -----
Error: 1.764131951631498e-09
----- 0.0s -----
Error: 1.763542201160817e-09
----- 0.0s -----
Error: 1.762956003403815e-09
----- 0.0s -----
Error: 1.76236625293

Error: 1.3121379538461042e-09
----- 0.0s -----
Error: 1.3117116282046482e-09
----- 0.0s -----
Error: 1.3112710917084769e-09
----- 0.0s -----
Error: 1.3108305552123056e-09
----- 0.0s -----
Error: 1.3103900187161344e-09
----- 0.0s -----
Error: 1.309953034933642e-09
----- 0.001s -----
Error: 1.309523156578507e-09
----- 0.0s -----
Error: 1.3090826200823358e-09
----- 0.0s -----
Error: 1.3086491890135221e-09
----- 0.0s -----
Error: 1.3082086525173509e-09
----- 0.0s -----
Error: 1.3077752214485372e-09
----- 0.0s -----
Error: 1.307334684952366e-09
----- 0.0s -----
Error: 1.3069012538835523e-09
----- 0.0s -----
Error: 1.3064678228147386e-09
----- 0.0s -----
Error: 1.3060272863185673e-09
----- 0.0s -----
Error: 1.3055938552497537e-09
----- 0.0s -----
Error: 1.30516042418094e-09
----- 0.0s -----
Error: 1.3047216640416082e-09
----- 0.0s -----
Error: 1.3042935620433127e-09
----- 0.0s -----
Error: 1.3038530255471414e-09
----- 0.0s -----
Error: 1.3034195944783278e-09
----- 0.0s -----
Error: 1.3029861

In [12]:
european_call_prices = vector_to_spreadsheet(vector=value_function)
european_call_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,68.773148
1,,,,,,,,,,,,,,,61.989658,55.448175
2,,,,,,,,,,,,,,55.466344,49.17303,43.116387
3,,,,,,,,,,,,,49.193278,43.138666,37.311697,31.703758
4,,,,,,,,,,,,43.160913,37.335898,31.72984,26.334456,21.141776
5,,,,,,,,,,,37.360066,31.755889,26.362316,21.171378,16.175411,11.367038
6,,,,,,,,,,31.781905,26.390141,21.200946,16.206656,11.399898,6.773577,2.320867
7,,,,,,,,,26.470299,21.333692,16.441297,11.83368,7.598235,3.91437,1.142575,-0.000003
8,,,,,,,,21.526807,16.744143,12.301778,8.293172,4.863169,2.212455,0.562495,-0.000003,-0.000003
9,,,,,,,17.072243,12.761073,8.904576,5.614028,3.01807,1.229702,0.276918,-0.000003,-0.000003,-0.000003


-----

In [13]:
p = generate_probabilities(states, actions, q)
r = generate_rewards(states, actions, put_payoff(K))
discount_factor = 1/R

In [14]:
mdp = MDP(states = states, 
          actions = actions, 
          probabilities = p, 
          rewards = r, 
          discount_factor = discount_factor)

In [15]:
mdp.value_iteration(theta=1e-9)

Error: 54.028085311431
----- 0.0s -----
Error: 1.1899381916725562
----- 0.0s -----
Error: 0.6190932491462617
----- 0.0s -----
Error: 0.5849693753671343
----- 0.0s -----
Error: 0.46955096344356484
----- 0.0s -----
Error: 0.44211296121782073
----- 0.001s -----
Error: 0.39306149751896546
----- 0.0s -----
Error: 0.37012835068744643
----- 0.0s -----
Error: 0.34433421629057115
----- 0.0s -----
Error: 0.3245821719201425
----- 0.0s -----
Error: 0.28273264424122146
----- 0.0s -----
Error: 0.27178134943374666
----- 0.0s -----
Error: 0.24106195611905434
----- 0.0s -----
Error: 0.20237885900167285
----- 0.0s -----
Error: 0.18598560852091595
----- 0.0s -----
Error: 0.15864054203021816
----- 0.0s -----
Error: 1.2365197042285558e-05
----- 0.0s -----
Error: 1.236107599567049e-05
----- 0.0s -----
Error: 1.2356956322179258e-05
----- 0.0s -----
Error: 1.235283802358822e-05
----- 0.0s -----
Error: 1.2348721099897375e-05
----- 0.0s -----
Error: 1.234460554222494e-05
----- 0.0s -----
Error: 1.23404913630054

Error: 8.535373954288161e-06
----- 0.0s -----
Error: 8.532529307103687e-06
----- 0.0s -----
Error: 8.529685603164694e-06
----- 0.0s -----
Error: 8.526842851352967e-06
----- 0.0s -----
Error: 8.524001044563079e-06
----- 0.0s -----
Error: 8.521160182795029e-06
----- 0.0s -----
Error: 8.518320267825175e-06
----- 0.0s -----
Error: 8.515481301429872e-06
----- 0.0s -----
Error: 8.512643283609123e-06
----- 0.0s -----
Error: 8.50980620548114e-06
----- 0.0s -----
Error: 8.506970075927711e-06
----- 0.0s -----
Error: 8.504134893172477e-06
----- 0.0s -----
Error: 8.501300651886368e-06
----- 0.0s -----
Error: 8.498467357398454e-06
----- 0.0s -----
Error: 8.495635007932378e-06
----- 0.0s -----
Error: 8.492803601711785e-06
----- 0.0s -----
Error: 8.489973136960316e-06
----- 0.0s -----
Error: 8.487143617230686e-06
----- 0.0s -----
Error: 8.484315040746537e-06
----- 0.0s -----
Error: 8.48148740750787e-06
----- 0.0s -----
Error: 8.478660717514686e-06
----- 0.0s -----
Error: 8.475834967214269e-06
----- 0

----- 0.0s -----
Error: 5.911418600845764e-06
----- 0.0s -----
Error: 5.9094484576860395e-06
----- 0.0s -----
Error: 5.907478970001989e-06
----- 0.0s -----
Error: 5.905510139569969e-06
----- 0.0s -----
Error: 5.903541964613623e-06
----- 0.0s -----
Error: 5.901574443356594e-06
----- 0.0s -----
Error: 5.8996075811279525e-06
----- 0.0s -----
Error: 5.897641372598628e-06
----- 0.0s -----
Error: 5.8956758195449765e-06
----- 0.0s -----
Error: 5.893710920190642e-06
----- 0.0s -----
Error: 5.891746674535625e-06
----- 0.0s -----
Error: 5.889783089685352e-06
----- 0.0s -----
Error: 5.887820154981682e-06
----- 0.0s -----
Error: 5.885857873977329e-06
----- 0.0s -----
Error: 5.883896250225007e-06
----- 0.0s -----
Error: 5.881935276619288e-06
----- 0.0s -----
Error: 5.879974963818313e-06
----- 0.0s -----
Error: 5.878015294058514e-06
----- 0.0s -----
Error: 5.8760562815507456e-06
----- 0.0s -----
Error: 5.874097922742294e-06
----- 0.0s -----
Error: 5.872140217633159e-06
----- 0.0s -----
Error: 5.8701

----- 0.0s -----
Error: 3.897047326617553e-06
----- 0.0s -----
Error: 3.895748527327214e-06
----- 0.0s -----
Error: 3.894450159691587e-06
----- 0.0s -----
Error: 3.893152225487029e-06
----- 0.0s -----
Error: 3.891854724713539e-06
----- 0.0s -----
Error: 3.890557655594762e-06
----- 0.0s -----
Error: 3.889261019907053e-06
----- 0.0s -----
Error: 3.8879648158740565e-06
----- 0.0s -----
Error: 3.886669041719415e-06
----- 0.0s -----
Error: 3.885373702772199e-06
----- 0.0s -----
Error: 3.884078797256052e-06
----- 0.0s -----
Error: 3.8827843162891895e-06
----- 0.0s -----
Error: 3.881490270529753e-06
----- 0.0s -----
Error: 3.880196658201385e-06
----- 0.0s -----
Error: 3.878903473975015e-06
----- 0.0s -----
Error: 3.877610723179714e-06
----- 0.0s -----
Error: 3.876318402262768e-06
----- 0.0s -----
Error: 3.875026511224178e-06
----- 0.0s -----
Error: 3.873735050063942e-06
----- 0.0s -----
Error: 3.872444018782062e-06
----- 0.0s -----
Error: 3.8711534191548935e-06
----- 0.0s -----
Error: 3.86986

Error: 2.569948442499026e-06
----- 0.0s -----
Error: 2.5690919365217724e-06
----- 0.0s -----
Error: 2.5682357183143267e-06
----- 0.0s -----
Error: 2.5673797807712617e-06
----- 0.0s -----
Error: 2.5665241309980047e-06
----- 0.0s -----
Error: 2.565668763665485e-06
----- 0.0s -----
Error: 2.5648136841027735e-06
----- 0.0s -----
Error: 2.5639588869807994e-06
----- 0.0s -----
Error: 2.5631043776286333e-06
----- 0.0s -----
Error: 2.562250148940848e-06
----- 0.0s -----
Error: 2.561396209799227e-06
----- 0.0s -----
Error: 2.560542551321987e-06
----- 0.0s -----
Error: 2.559689180614555e-06
----- 0.0s -----
Error: 2.558836094124217e-06
----- 0.0s -----
Error: 2.557983290074617e-06
----- 0.0s -----
Error: 2.557130770242111e-06
----- 0.0s -----
Error: 2.556278536403056e-06
----- 0.0s -----
Error: 2.5554265867810955e-06
----- 0.0s -----
Error: 2.5545749195998724e-06
----- 0.0s -----
Error: 2.5537235366357436e-06
----- 0.0s -----
Error: 2.5528724361123523e-06
----- 0.0s -----
Error: 2.55202161980605

...

Error: 6.451472245316836e-09
----- 0.0s -----
Error: 6.449322853541162e-09
----- 0.0s -----
Error: 6.4471734617654874e-09
----- 0.0s -----
Error: 6.445024069989813e-09
----- 0.0s -----
Error: 6.442876454570978e-09
----- 0.0s -----
Error: 6.440730615508983e-09
----- 0.0s -----
Error: 6.438583000090148e-09
----- 0.0s -----
Error: 6.436437161028152e-09
----- 0.0s -----
Error: 6.434291321966157e-09
----- 0.0s -----
Error: 6.43214903561784e-09
----- 0.0s -----
Error: 6.430004972912684e-09
----- 0.0s -----
Error: 6.427860910207528e-09
----- 0.0s -----
Error: 6.425718623859211e-09
----- 0.0s -----
Error: 6.4235763375108945e-09
----- 0.0s -----
Error: 6.421437603876257e-09
----- 0.0s -----
Error: 6.419298870241619e-09
----- 0.0s -----
Error: 6.417160136606981e-09
----- 0.0s -----
Error: 6.4150196266155035e-09
----- 0.0s -----
Error: 6.412879116624026e-09
----- 0.0s -----
Error: 6.410743935703067e-09
----- 0.0s -----
Error: 6.408605202068429e-09
----- 0.0s -----
Error: 6.40647002114747e-09
----

Error: 4.570587464058917e-09
----- 0.0s -----
Error: 4.569066902604391e-09
----- 0.0s -----
Error: 4.567546341149864e-09
----- 0.0s -----
Error: 4.566022226981659e-09
----- 0.0s -----
Error: 4.564498112813453e-09
----- 0.0s -----
Error: 4.5629775513589266e-09
----- 0.0s -----
Error: 4.5614569899044e-09
----- 0.0s -----
Error: 4.559934652093034e-09
----- 0.0s -----
Error: 4.5584194197090255e-09
----- 0.0s -----
Error: 4.556898858254499e-09
----- 0.0s -----
Error: 4.555378296799972e-09
----- 0.0s -----
Error: 4.5538612880591245e-09
----- 0.0s -----
Error: 4.552344279318277e-09
----- 0.0s -----
Error: 4.550827270577429e-09
----- 0.0s -----
Error: 4.549308485479742e-09
----- 0.0s -----
Error: 4.547793253095733e-09
----- 0.0s -----
Error: 4.5462762443548854e-09
----- 0.0s -----
Error: 4.544762788327716e-09
----- 0.0s -----
Error: 4.543249332300547e-09
----- 0.0s -----
Error: 4.541734099916539e-09
----- 0.0s -----
Error: 4.5402188675325306e-09
----- 0.0s -----
Error: 4.5387054115053616e-09
-

Error: 3.0637608006145456e-09
----- 0.001s -----
Error: 3.06274117178873e-09
----- 0.0s -----
Error: 3.0617179902492353e-09
----- 0.0s -----
Error: 3.06069658506658e-09
----- 0.0s -----
Error: 3.059678732597604e-09
----- 0.0s -----
Error: 3.0586555510581093e-09
----- 0.0s -----
Error: 3.057637698589133e-09
----- 0.0s -----
Error: 3.0566198461201566e-09
----- 0.0s -----
Error: 3.055600217294341e-09
----- 0.0s -----
Error: 3.054580588468525e-09
----- 0.0s -----
Error: 3.053564512356388e-09
----- 0.0s -----
Error: 3.052548436244251e-09
----- 0.0s -----
Error: 3.051530583775275e-09
----- 0.0s -----
Error: 3.050510954949459e-09
----- 0.0s -----
Error: 3.049494878837322e-09
----- 0.0s -----
Error: 3.0484805790820246e-09
----- 0.0s -----
Error: 3.0474645029698877e-09
----- 0.0s -----
Error: 3.0464484268577507e-09
----- 0.0s -----
Error: 3.045432350745614e-09
----- 0.0s -----
Error: 3.0444180509903163e-09
----- 0.0s -----
Error: 3.0434019748781793e-09
----- 0.0s -----
Error: 3.0423876751228818

Error: 2.1204806799346443e-09
----- 0.001s -----
Error: 2.119776354447822e-09
----- 0.0s -----
Error: 2.1190693644257408e-09
----- 0.0s -----
Error: 2.118364150760499e-09
----- 0.0s -----
Error: 2.117658937095257e-09
----- 0.0s -----
Error: 2.1169501707163363e-09
----- 0.0s -----
Error: 2.116243180694255e-09
----- 0.0s -----
Error: 2.115537967029013e-09
----- 0.0s -----
Error: 2.1148345297206106e-09
----- 0.0s -----
Error: 2.114134645125887e-09
----- 0.0s -----
Error: 2.1134241023901268e-09
----- 0.0s -----
Error: 2.1127206650817243e-09
----- 0.0s -----
Error: 2.1120172277733218e-09
----- 0.0s -----
Error: 2.1113102377512405e-09
----- 0.0s -----
Error: 2.110610353156517e-09
----- 0.0s -----
Error: 2.1099069158481143e-09
----- 0.0s -----
Error: 2.109203478539712e-09
----- 0.0s -----
Error: 2.10849826487447e-09
----- 0.0s -----
Error: 2.107796603922907e-09
----- 0.0s -----
Error: 2.1070949429713437e-09
----- 0.0s -----
Error: 2.1063932820197806e-09
----- 0.0s -----
Error: 2.1056898447113

Error: 1.4443291007637526e-09
----- 0.0s -----
Error: 1.4438477080602752e-09
----- 0.0s -----
Error: 1.443371644427316e-09
----- 0.0s -----
Error: 1.442888475366999e-09
----- 0.0s -----
Error: 1.4424035299498428e-09
----- 0.0s -----
Error: 1.4419221372463653e-09
----- 0.0s -----
Error: 1.4414425208997272e-09
----- 0.0s -----
Error: 1.4409629045530892e-09
----- 0.0s -----
Error: 1.4404797354927723e-09
----- 0.0s -----
Error: 1.4400001191461342e-09
----- 0.0s -----
Error: 1.4395205027994962e-09
----- 0.0s -----
Error: 1.439044439166537e-09
----- 0.0s -----
Error: 1.43856127010622e-09
----- 0.0s -----
Error: 1.4380852064732608e-09
----- 0.0s -----
Error: 1.437602037412944e-09
----- 0.0s -----
Error: 1.4371224210663058e-09
----- 0.0s -----
Error: 1.4366445810765072e-09
----- 0.0s -----
Error: 1.4361667410867085e-09
----- 0.0s -----
Error: 1.4356871247400704e-09
----- 0.0s -----
Error: 1.4352110611071112e-09
----- 0.0s -----
Error: 1.434731444760473e-09
----- 0.0s -----
Error: 1.43425538112

In [16]:
american_put_prices = vector_to_spreadsheet(vector=mdp.value_star)
american_put_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,-0.000003
1,,,,,,,,,,,,,,,-0.000003,-0.000003
2,,,,,,,,,,,,,,-0.000003,-0.000003,-0.000003
3,,,,,,,,,,,,,-0.000003,-0.000003,-0.000003,-0.000003
4,,,,,,,,,,,,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003
5,,,,,,,,,,,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003
6,,,,,,,,,,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003,-0.000003
7,,,,,,,,,0.052363,0.103209,0.203427,0.400954,0.790276,1.557625,3.070057,6.051036
8,,,,,,,,0.266997,0.475439,0.836935,1.452195,2.473197,4.107806,6.585014,9.999997,13.798939
9,,,,,,,0.77301,1.264515,2.031009,3.190985,4.880275,7.219134,10.242882,13.798939,17.453562,20.969348


In [17]:
mdp.policy_star

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

-----

In [18]:
mdp.policy_iteration(theta=1e-9)

Error: 4.109233062231427
----- 2.577s -----
Error: 0.9082513369772443
----- 2.011s -----
Error: 0.4679459450989505
----- 2.745s -----
Error: 0.1810680132722915
----- 2.714s -----
Error: 0.028968803746238336
----- 2.412s -----
Error: 1.2789413972313923e-10
----- 2.835s -----


In [19]:
american_put_prices = vector_to_spreadsheet(vector=mdp.value_star)
american_put_prices

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,,,,,,,,,,,,,,,,0.000003
1,,,,,,,,,,,,,,,0.000003,0.000003
2,,,,,,,,,,,,,,0.000003,0.000003,0.000003
3,,,,,,,,,,,,,0.000003,0.000003,0.000003,0.000003
4,,,,,,,,,,,,0.000003,0.000003,0.000003,0.000003,0.000003
5,,,,,,,,,,,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
6,,,,,,,,,,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
7,,,,,,,,,0.052369,0.103215,0.203433,0.40096,0.790282,1.557631,3.070063,6.051042
8,,,,,,,,0.267003,0.475445,0.836941,1.452201,2.473203,4.107812,6.58502,10.000003,13.798945
9,,,,,,,0.773016,1.264521,2.031015,3.190991,4.880281,7.21914,10.242888,13.798945,17.453568,20.969354


In [20]:
mdp.policy_star

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.